In [2]:
# Rafael Vargas Serenato
#
# Olá, esse é o TDE 2 de AI
#
# Nesse programa vamos fazer um jogo da velha com tabuleiro 4x4, com 2 modos de jogo 'Humano x Computador' e 'Computador x Computador'
# também podendo escolher os modos de jogada do computador 'Aleatório', 'Minimax' e 'Minimax e poda Alfa-Beta'
#
#
#
#     Como jogar:
#
# O código inicia pedindo para selecionar o modo de jogo (1 para'Humano x Computador' e 2 para 'Computador x Computador')
# depois você vai selecionar o método de jogada do computador (1 para 'Aleatório', 2 para 'Minimax' e 3 para 'Minimax com poda Alfa-Beta')
# se selecionar o modo de jogo 'Humano x Computador', você começara o jogo como jogador 'X' fazendo suas jogadas informando o número de LINHA e COLUNA separado por um ESPAÇO
#
#
#         Exemplo da jogada por posição:
#                                                 0:0 | 0:1 | 0:2 | 0:3
#                                                 ---------------------
#                                                 1:0 | 1:1 | 1:2 | 1:3
#                                                 ---------------------
#                                                 2:0 | 2:1 | 2:2 | 2:3
#                                                 ---------------------
#                                                 3:0 | 3:1 | 3:2 | 3:3
#                                                 ---------------------

In [3]:
# Importar bibliotecas :)
import random, time, threading
import numpy as np

In [4]:
# Classe do jogo da velha 4x4
class JogoVelha4x4:
    # Função padrão do programa
    def __init__(self):
        self.tabuleiro = [[" " for _ in range(4)] for _ in range(4)]
        self.jogador_atual = "X"
        self.jogadas = 0

    # Função para montar o tabuleiro
    def exibir_tabuleiro(self):
        for linha in self.tabuleiro:
            print(" | ".join(linha))
            print("-" * 14)

    # Função de jogada
    def jogada(self, linha, coluna):
        if self.tabuleiro[linha][coluna] == " ":
            self.tabuleiro[linha][coluna] = self.jogador_atual
            self.jogadas += 1
            self.jogador_atual = "O" if self.jogador_atual == "X" else "X"
            return True
        #return False, print("\nPosição já ocupada, tente novamente\n") # Se a posição escolhida já estiver em uso, ele retorna

    # Verifica se a jogada resultou em vitória
    def verificar_vitoria(self):
        # Verifica linhas e colunas
        for i in range(4):
            if all([self.tabuleiro[i][j] == self.tabuleiro[i][0] != " " for j in range(4)]) or \
               all([self.tabuleiro[j][i] == self.tabuleiro[0][i] != " " for j in range(4)]):
                return True
        # Verifica diagonais
        if all([self.tabuleiro[i][i] == self.tabuleiro[0][0] != " " for i in range(4)]) or \
           all([self.tabuleiro[i][3-i] == self.tabuleiro[0][3] != " " for i in range(4)]):
            return True
        return False

    # Se tiver dado o número máximo de jogadas e não resultar em vitório, então empate
    def verificar_empate(self):
        return self.jogadas == 16

    # Função para definir o estado do jogo
    def estado_jogo(self):
        if self.verificar_vitoria():
            return "Vitoria"
        if self.verificar_empate():
            return "Empate"
        return "Em andamento"

In [12]:
# Função de Aleatório
class AgenteAleatorio:
    def __init__(self, simbolo):
        self.simbolo = simbolo

    def melhor_movimento(self, jogo):
        jogadas_possiveis = [(i, j) for i in range(4) for j in range(4) if jogo.tabuleiro[i][j] == " "]
        return random.choice(jogadas_possiveis)

In [6]:
# Função de Minimax

# Função para contar as possíveis vitórias de um jogador
def count_possible_wins(board, player):
    # definir quem é o oponente
    if player == 1:
        opponent = 2
    else:
        opponent = 1

    # contagem das horizontais/verticais
    count = 0
    for i in range(4):
        # horizontal
        if opponent not in board[i, :]:
            count += 1
        # vertical
        if opponent not in board[:, i]:
            count += 1

    # contagem da diagonal (esquerda -> direita)
    if opponent not in np.diag(board):
        count += 1

    # contagem da diagonal (direita -> esquerda)
    if opponent not in np.diag(np.fliplr(board)):
        count += 1

    return count

# Função de avaliação heurística
def evaluation_function(board):
    # O valor total será a diferença entre as chances de vitória do jogador e as chances de vitória do oponente
    player_score = count_possible_wins(board, 1)  # Contagem de vitórias possíveis do jogador
    opponent_score = count_possible_wins(board, 2)  # Contagem de vitórias possíveis do oponente

    # Penaliza mais fortemente se o oponente estiver muito perto de ganhar
    for i in range(4):
        if np.count_nonzero(board[i, :] == 2) == 3 and np.count_nonzero(board[i, :] == 0) == 1:
            opponent_score += 10  # Penaliza se o oponente estiver a uma jogada de ganhar na linha
        if np.count_nonzero(board[:, i] == 2) == 3 and np.count_nonzero(board[:, i] == 0) == 1:
            opponent_score += 10  # Penaliza se o oponente estiver a uma jogada de ganhar na coluna

    # Penaliza nas diagonais
    if np.count_nonzero(np.diag(board) == 2) == 3 and np.count_nonzero(np.diag(board) == 0) == 1:
        opponent_score += 10
    if np.count_nonzero(np.diag(np.fliplr(board)) == 2) == 3 and np.count_nonzero(np.diag(np.fliplr(board)) == 0) == 1:
        opponent_score += 10

    return player_score - opponent_score  # Maximiza a chance do jogador e minimiza a do oponente


# Função Minimax com profundidade e tempo limite
def minimax(jogo, profundidade, max_profundidade, is_maximizing, jogador, oponente, tempo_limite, start_time):
    # Transformando o tabuleiro para um formato adequado (np.array)
    board = np.array([[1 if cell == 'X' else (2 if cell == 'O' else 0) for cell in row] for row in jogo.tabuleiro])

    if profundidade >= max_profundidade or jogo.estado_jogo() != "Em andamento":
        return evaluation_function(board)

    if time.time() - start_time > tempo_limite:
        return evaluation_function(board)  # Retorna avaliação do estado atual se o tempo limite for atingido

    melhor_valor = -float('inf') if is_maximizing else float('inf')

    for i in range(4):
        for j in range(4):
            if jogo.tabuleiro[i][j] == " ":
                jogo.tabuleiro[i][j] = 'X' if is_maximizing else 'O'
                valor = minimax(jogo, profundidade + 1, max_profundidade, not is_maximizing, jogador, oponente, tempo_limite, start_time)
                jogo.tabuleiro[i][j] = " "
                if is_maximizing:
                    melhor_valor = max(melhor_valor, valor)
                else:
                    melhor_valor = min(melhor_valor, valor)

    return melhor_valor

# Função para escolher a melhor jogada usando Minimax
def jogada_com_tempo_limite(jogo, simbolo, profundidade_maxima, limite_segundos):
    start_time = time.time()
    melhor_valor = -float('inf')
    melhor_movimento = None

    for i in range(4):
        for j in range(4):
            if jogo.tabuleiro[i][j] == " ":
                jogo.tabuleiro[i][j] = simbolo
                valor = minimax(jogo, 0, profundidade_maxima, True, simbolo, "O" if simbolo == "X" else "X", limite_segundos, start_time)
                jogo.tabuleiro[i][j] = " "
                if valor > melhor_valor:
                    melhor_valor = valor
                    melhor_movimento = (i, j)
                if time.time() - start_time > limite_segundos:
                    return melhor_movimento if melhor_movimento else None

    return melhor_movimento

# Classe Minimax
class AgenteMinimax:
    def __init__(self, simbolo):
        self.simbolo = simbolo

    def jogar(self, jogo):
        # Limite de 60 segundos para cada busca para jogada
        return jogada_com_tempo_limite(jogo, self.simbolo, profundidade_maxima=7, limite_segundos=60) or AgenteAleatorio(self.simbolo).jogar(jogo)


In [7]:
# Função Minimax com poda Alfa-Beta

class AgenteMinimaxAlfaBeta:
    def __init__(self, jogador, oponente, max_profundidade, limite_segundos):
        self.jogador = jogador  # O jogador controlado pelo algoritmo
        self.oponente = oponente  # O oponente do jogador
        self.max_profundidade = max_profundidade  # Profundidade máxima para o algoritmo
        self.limite_segundos = limite_segundos  # Limite de tempo em segundos
        self.evaluation_function = evaluation_function  # Função de avaliação heurística

    def melhor_movimento(self, jogo):
        start_time = time.time()
        melhor_valor = -float('inf')
        melhor_movimento = None

        # Verifica todos os movimentos possíveis no tabuleiro
        for i in range(4):
            for j in range(4):
                if jogo.tabuleiro[i][j] == " ":
                    jogo.tabuleiro[i][j] = self.jogador  # Faz a jogada temporária
                    valor = self.minimax_alfa_beta(jogo, 1, self.max_profundidade, -float('inf'), float('inf'), False, start_time)
                    jogo.tabuleiro[i][j] = " "  # Desfaz a jogada

                    if valor > melhor_valor:
                        melhor_valor = valor
                        melhor_movimento = (i, j)

        return melhor_movimento

    def minimax_alfa_beta(self, jogo, profundidade, max_profundidade, alpha, beta, is_maximizing, start_time):
        # Transformando o tabuleiro para um formato adequado (np.array) para a função de avaliação
        board = np.array([[1 if cell == 'X' else (2 if cell == 'O' else 0) for cell in row] for row in jogo.tabuleiro])

        # Condição de parada por profundidade ou fim de jogo
        if profundidade >= max_profundidade or jogo.estado_jogo() != "Em andamento":
            return self.evaluation_function(board)  # Retorna avaliação do estado atual

        # Condição de parada por tempo limite
        if time.time() - start_time > self.limite_segundos:
            return self.evaluation_function(board)  # Retorna avaliação do estado atual

        # Maximizing (jogador atual)
        if is_maximizing:
            melhor_valor = -float('inf')
            for i in range(4):
                for j in range(4):
                    if jogo.tabuleiro[i][j] == " ":
                        # Verificação adicional de tempo no loop
                        if time.time() - start_time > self.limite_segundos:
                            return self.evaluation_function(board)  # Retorna avalação do estado atual

                        jogo.tabuleiro[i][j] = self.jogador
                        valor = self.minimax_alfa_beta(jogo, profundidade + 1, max_profundidade, alpha, beta, False, start_time)
                        jogo.tabuleiro[i][j] = " "
                        melhor_valor = max(melhor_valor, valor)
                        alpha = max(alpha, valor)
                        if beta <= alpha:
                            break  # Poda beta
            return melhor_valor
        # Minimizing (oponente)
        else:
            melhor_valor = float('inf')
            for i in range(4):
                for j in range(4):
                    if jogo.tabuleiro[i][j] == " ":
                        # Verificação adicional de tempo no loop
                        if time.time() - start_time > self.limite_segundos:
                            return self.evaluation_function(board)  # Retorna avaliação do estado atual

                        jogo.tabuleiro[i][j] = self.oponente
                        valor = self.minimax_alfa_beta(jogo, profundidade + 1, max_profundidade, alpha, beta, True, start_time)
                        jogo.tabuleiro[i][j] = " "
                        melhor_valor = min(melhor_valor, valor)
                        beta = min(beta, valor)
                        if beta <= alpha:
                            break  # Poda alfa
            return melhor_valor


In [10]:
# Função principal para selecionar modo de jogo e iniciar
def iniciar_jogo():
    nr_jogada = 1 # Variável para contar o número de jogadas
    jogo = JogoVelha4x4() # Criar uma instância do jogo da velha

    print("\nSelecione o modo de jogo:")
    print("1. Humano X Computador")
    print("2. Computador X Computador")
    modo = int(input("Modo: "))

    print("\n\nSelecione a estratégia do computador:")
    print("1. Aleatório")
    print("2. Minimax")
    print("3. Minimax e poda Alfa-Beta")
    estrategia = int(input("Estratégia: "))

    print("\n")
    jogo.exibir_tabuleiro()

    if estrategia == 1:
        # Aleatório
        agenteX = AgenteAleatorio("X")
        agenteO = AgenteAleatorio("O")
    elif estrategia == 2:
        # Minimax
        agenteX = AgenteMinimax("X")
        agenteO = AgenteMinimax("O")
    elif estrategia == 3:
        # Minimax com poda Alfa-beta
        agenteX = AgenteMinimaxAlfaBeta(jogador="X", oponente="O", max_profundidade=6, limite_segundos=30)
        agenteO = AgenteMinimaxAlfaBeta(jogador="O", oponente="X", max_profundidade=6, limite_segundos=30)

    else:
        # Se for passado um valor inválido, inicia como aleatório
        print("Opção inválida. Iniciando o jogo com estratégia aleatória.")
        agenteX = AgenteAleatorio("X")
        agenteO = AgenteAleatorio("O")

    while jogo.estado_jogo() == "Em andamento":
        if modo == 1:
            # Humano X Computador
            if jogo.jogador_atual == "X":
                player = "X"
                i = 0
                while i == 0:
                  linha, coluna = map(int, input(f"\n\n{nr_jogada} - Sua jogada (linha coluna): ").split())
                  if linha < 0 or linha > 3 or coluna < 0 or coluna > 3 or jogo.tabuleiro[linha][coluna] != " ":
                      print("\nJogada inválida. Tente novamente.")
                  elif True == jogo.jogada(linha, coluna):
                      i = 1
                      nr_jogada += 1

            else:
                print(f"\n\n{nr_jogada} - Jogada do Computador O:")
                player = "O"
                tempo_inicial = time.time()
                linha, coluna = agenteO.melhor_movimento(jogo)
                jogo.jogada(linha, coluna)
                tempo_gasto = time.time() - tempo_inicial
                print(f"Computador O jogou em ({linha}, {coluna}) em {tempo_gasto:.2f} segundos")
                nr_jogada += 1
        elif modo == 2:
            # Computador X Computador
            if jogo.jogador_atual == "X":
                player = "X"
                print(f"\n\n{nr_jogada} - Jogada do Computador X:")
                tempo_inicial = time.time()
                linha, coluna = agenteX.melhor_movimento(jogo)
                jogo.jogada(linha, coluna)
                tempo_gasto = time.time() - tempo_inicial
                print(f"Computador X jogou em ({linha}, {coluna}) em {tempo_gasto:.2f} segundos")
                nr_jogada += 1
            else:
                print(f"\n\n{nr_jogada} - Jogada do Computador O:")
                player = "O"
                tempo_inicial = time.time()
                linha, coluna = agenteO.melhor_movimento(jogo)
                jogo.jogada(linha, coluna)
                tempo_gasto = time.time() - tempo_inicial
                print(f"Computador O jogou em ({linha}, {coluna}) em {tempo_gasto:.2f} segundos")
                nr_jogada += 1
        else:
            # Se iniciar com um valor inválido inicia como Humano x Computador
            print("Modo inválido. Iniciando o jogo com modo 'Humano X Computador'.")
            modo = 1

        print("\n")
        jogo.exibir_tabuleiro() # Exibe o tabuleiro após uma jogada

        if jogo.estado_jogo() == "Vitoria":
            print(f"\nO jogador {player} venceu! total jogadas: {nr_jogada - 1}")
        elif jogo.estado_jogo() == "Empate":
            print(f"\nO jogo terminou em empate! total jogadas: {nr_jogada - 1}")

In [13]:
# Iniciar jogo :O (não creio)
iniciar_jogo()


Selecione o modo de jogo:
1. Humano X Computador
2. Computador X Computador
Modo: 1


Selecione a estratégia do computador:
1. Aleatório
2. Minimax
3. Minimax e poda Alfa-Beta
Estratégia: 1


  |   |   |  
--------------
  |   |   |  
--------------
  |   |   |  
--------------
  |   |   |  
--------------


1 - Sua jogada (linha coluna): 0 0


X |   |   |  
--------------
  |   |   |  
--------------
  |   |   |  
--------------
  |   |   |  
--------------


2 - Jogada do Computador O:
Computador O jogou em (2, 0) em 0.00 segundos


X |   |   |  
--------------
  |   |   |  
--------------
O |   |   |  
--------------
  |   |   |  
--------------


3 - Sua jogada (linha coluna): 0 1


X | X |   |  
--------------
  |   |   |  
--------------
O |   |   |  
--------------
  |   |   |  
--------------


4 - Jogada do Computador O:
Computador O jogou em (1, 0) em 0.00 segundos


X | X |   |  
--------------
O |   |   |  
--------------
O |   |   |  
--------------
  |   |   |  
--------